In [ ]:
%pylab inline
import pandas

In [ ]:
TRAINFILE='../data/training_input.csv'
LABELFILE='../data/challenge_output_data_training_file_prediction_of_trading_activity_within_the_order_book.csv'
CHUNKSIZE=8
NCHUNKS=10
LASTROW=7
DAY=4679
USEFEATURES = ['ID', 'bid_1', 'ask_1', 'bid_size_1', 'ask_size_1', 'nb_trade']
NEWFEATURES = ['bid_1', 'spread', 'bid_plus_ask', 'bid_pct', 'nb_trade']
TRAINPICKLE='train.pkl'

In [ ]:
train_all = pandas.read_csv(TRAINFILE, nrows=CHUNKSIZE*NCHUNKS, usecols=USEFEATURES)

In [ ]:
train_all['spread'] = train_all['ask_1']-train_all['bid_1']
train_all['bid_plus_ask'] = train_all['bid_size_1']-train_all['ask_size_1']
train_all['bid_pct'] = train_all['bid_size_1']/train_all['bid_plus_ask']

In [ ]:
train_diff = train_all[NEWFEATURES].diff()

In [ ]:
past_diff_feat = np.hstack([train_diff.iloc[i::8, :].values for i in range(1,8)])

In [ ]:
fut_diff_feat = np.hstack([train_diff.iloc[i+7::8, :].values for i in range(1,8)])
# fix end
fut_diff_feat = np.r_[fut_diff_feat, np.zeros((1, B.shape[1]))]

In [ ]:
pres_orig_feat = train_all.iloc[7::8][NEWFEATURES].values

In [ ]:
time_feat = (train_all.iloc[7::8, 0].values-1)%DAY

In [ ]:
train=np.c_[past_diff_feat,fut_diff_feat,pres_orig_feat,time_feat]

In [ ]:
np.save(TRAINPICKLE, train, allow_pickle=True)

In [ ]:
label = pandas.read_csv(LABELFILE, sep=';')['TARGET'].values

In [ ]:
label.astype('float64').round().astype('int').mean()

In [ ]:
#import xgboost as xgb
def pct_eval(y_prob, dtrain):
    y_true = dtrain.get_label()
    return 'PCT', 1-(y_prob.round().astype('int')==y_true).mean()

In [ ]:
dtrain = xgb.DMatrix(data=np.load(TRAINPICKLE), label = label)
prior=label.mean()
params={}
params['bst:eta'] = 0.1
params['bst:max_depth'] = 4
params['min_child_weight'] = 14
params['objective'] = 'binary:logistic'
params['nthread'] = 4
params['eval_metric'] = 'auc'
params['learning_rate'] = 0.1
params['base_score'] = prior
res = xgb.cv(params, dtrain, num_boost_round=40, feval=pct_eval, verbose_eval=True, nfold=4, seed=0, early_stopping_rounds=10, show_stdv=True)